In [1]:
import torch
import sys
import numpy as np
import os
import yaml
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

Using device: cuda


In [3]:
# def get_stl10_data_loaders(download, shuffle=False, batch_size=256):
#   train_dataset = datasets.STL10('./data', split='train', download=download,
#                                   transform=transforms.ToTensor())

#   train_loader = DataLoader(train_dataset, batch_size=batch_size,
#                             num_workers=0, drop_last=False, shuffle=shuffle)
  
#   test_dataset = datasets.STL10('./data', split='test', download=download,
#                                   transform=transforms.ToTensor())

#   test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
#                             num_workers=10, drop_last=False, shuffle=shuffle)
#   return train_loader, test_loader

# def get_cifar10_data_loaders(download, shuffle=False, batch_size=256):
#   train_dataset = datasets.CIFAR10('./data', train=True, download=download,
#                                   transform=transforms.ToTensor())

#   train_loader = DataLoader(train_dataset, batch_size=batch_size,
#                             num_workers=0, drop_last=False, shuffle=shuffle)
  
#   test_dataset = datasets.CIFAR10('./data', train=False, download=download,
#                                   transform=transforms.ToTensor())

#   test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
#                             num_workers=10, drop_last=False, shuffle=shuffle)
#   return train_loader, test_loader

In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def get_custom_data_loaders(data_dir, batch_size=256, shuffle=True, num_workers=0):
    # Transformations for the grayscale images
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor()
    ])

    # Load the training dataset
    train_dataset = datasets.ImageFolder(root=f'{data_dir}/train', transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                              shuffle=shuffle, num_workers=num_workers, drop_last=True)

    # Load the test dataset
    test_dataset = datasets.ImageFolder(root=f'{data_dir}/test', transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size,
                             shuffle=False, num_workers=num_workers, drop_last=False)

    return train_loader, test_loader

In [5]:
# train_loader, test_loader = get_custom_data_loaders(data_dir='/path/to/your/dataset', batch_size=64, shuffle=True, num_workers=4)
train_loader, test_loader = get_custom_data_loaders(data_dir='D:\_workspace\ISRO_SimCLR\Custom_SimCLR\Data_SIMCLR_balanced', batch_size=64, shuffle=True, num_workers=4)

In [6]:
# with open(os.path.join('./config.yml')) as file:
#   config = yaml.load(file)

In [7]:
# if config.arch == 'resnet18':
#   model = torchvision.models.resnet18(pretrained=False, num_classes=10).to(device)
# elif config.arch == 'resnet50':
#   model = torchvision.models.resnet50(pretrained=False, num_classes=10).to(device)

In [8]:
# import torch
# import torchvision.models as models

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Adjust this path to where your trained model's weights are saved
# # model_path = 'path/to/your/trained_model.pth'
# model_path = 'D:\_workspace\ISRO_SimCLR\Custom_SimCLR\runs\test_run1\checkpoint_0005.pth.tar'

# # Load the model (assuming it's a ResNet18 trained with SimCLR)
# model = models.resnet18(pretrained=False)
# model.fc = torch.nn.Linear(model.fc.in_features, 4)  # Adjust for 4 classes

# # Modify the first layer to accept grayscale images
# model.conv1 = torch.nn.Conv2d(1, model.conv1.out_channels, kernel_size=model.conv1.kernel_size,
#                               stride=model.conv1.stride, padding=model.conv1.padding, bias=False)

# # Load the trained weights
# model.load_state_dict(torch.load(model_path))
# model = model.to(device)

In [9]:
# import torch
# import torchvision.models as models

# model_path = 'D:\\_workspace\\ISRO_SimCLR\\Custom_SimCLR\\runs\\test_run1\\checkpoint_0005.pth.tar'

# # Load the checkpoint
# checkpoint = torch.load(model_path, map_location='cpu')

# # Extract the state dictionary and remove 'backbone.' prefix from each key
# state_dict = {k.replace("backbone.", ""): v for k, v in checkpoint['state_dict'].items()}

# # Load the model (assuming it's a ResNet18 trained with SimCLR)
# model = models.resnet18(pretrained=False)

# # Modify the first layer to accept grayscale images
# model.conv1 = torch.nn.Conv2d(1, model.conv1.out_channels, kernel_size=model.conv1.kernel_size,
#                               stride=model.conv1.stride, padding=model.conv1.padding, bias=False)

# # Replace the fully connected layer for 4 classes
# model.fc = torch.nn.Linear(model.fc.in_features, 4)

# # Load the trained weights into the model
# model.load_state_dict(state_dict)

# # Move model to the appropriate device (GPU or CPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

In [10]:
# checkpoint = torch.load('checkpoint_0040.pth.tar', map_location=device)
checkpoint = torch.load('D:/_workspace/ISRO_SimCLR/Custom_SimCLR/runs/test_run1/checkpoint_0005.pth.tar', map_location=device)
state_dict = checkpoint['state_dict']

for k in list(state_dict.keys()):

  if k.startswith('backbone.'):
    if k.startswith('backbone') and not k.startswith('backbone.fc'):
      # remove prefix
      state_dict[k[len("backbone."):]] = state_dict[k]
  del state_dict[k]

In [11]:
 # Load the model (assuming it's a ResNet18 trained with SimCLR)
model = torchvision.models.resnet18(pretrained=False, num_classes=4).to(device)

# Modify the first layer to accept grayscale images
model.conv1 = torch.nn.Conv2d(1, model.conv1.out_channels, kernel_size=model.conv1.kernel_size,
                              stride=model.conv1.stride, padding=model.conv1.padding, bias=False)

# Move model to the appropriate device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

log = model.load_state_dict(state_dict, strict=False)
assert log.missing_keys == ['fc.weight', 'fc.bias']

d:\anaconda3\envs\torch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\anaconda3\envs\torch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [12]:
# if config.dataset_name == 'cifar10':
#   train_loader, test_loader = get_cifar10_data_loaders(download=True)
# elif config.dataset_name == 'stl10':
#   train_loader, test_loader = get_stl10_data_loaders(download=True)
# print("Dataset:", config.dataset_name)

In [13]:
# freeze all layers but the last fc
for name, param in model.named_parameters():
    if name not in ['fc.weight', 'fc.bias']:
        param.requires_grad = False

parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
assert len(parameters) == 2  # fc.weight, fc.bias

In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=0.0008)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [15]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [17]:
epochs = 5
for epoch in range(epochs):
  top1_train_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(train_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
    loss = criterion(logits, y_batch)
    top1 = accuracy(logits, y_batch, topk=(1,))
    top1_train_accuracy += top1[0]

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  top1_train_accuracy /= (counter + 1)
  top1_accuracy = 0
  top3_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(test_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
  
    # top1, top5 = accuracy(logits, y_batch, topk=(1,5))
    # top1_accuracy += top1[0]
    # top5_accuracy += top5[0]

    top1, top3 = accuracy(logits, y_batch, topk=(1,3))
    top1_accuracy += top1[0]
    top3_accuracy += top3[0]
  
  top1_accuracy /= (counter + 1)
  top3_accuracy /= (counter + 1)
  print(f"Epoch {epoch}\tTop1 Train accuracy {top1_train_accuracy.item()}\tTop1 Test accuracy: {top1_accuracy.item()}\tTop5 test acc: {top3_accuracy.item()}")

Epoch 0	Top1 Train accuracy 30.694730758666992	Top1 Test accuracy: 25.281761169433594	Top5 test acc: 75.82262420654297
Epoch 1	Top1 Train accuracy 31.15960693359375	Top1 Test accuracy: 25.308361053466797	Top5 test acc: 75.9378890991211
Epoch 2	Top1 Train accuracy 31.224172592163086	Top1 Test accuracy: 25.92015838623047	Top5 test acc: 76.34477233886719
Epoch 3	Top1 Train accuracy 31.94731330871582	Top1 Test accuracy: 24.926109313964844	Top5 test acc: 75.6590805053711
Epoch 4	Top1 Train accuracy 32.889976501464844	Top1 Test accuracy: 25.98222541809082	Top5 test acc: 76.12212371826172
